In [240]:
import polars as pl
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [241]:
thepath = r'C:\Users\MARKETING SUPPORT\Documents\asteros\astrs_work\on_prog_q4_explore.parquet'
masters = pl.read_parquet(thepath)

In [242]:
revenue = masters.filter([pl.col('Status Pesanan')!="Batal",pl.col('Waktu Pengiriman Diatur').is_not_null()]).sort(by='Waktu Pengiriman Diatur')['No. Pesanan','Status Pesanan','Alasan Pembatalan','Status Pembatalan/ Pengembalian','No. Resi',
       'Opsi Pengiriman','Antar ke counter/ pick-up','Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan)','Waktu Pengiriman Diatur',
       'Waktu Pesanan Dibuat','Waktu Pembayaran Dilakukan','Metode Pembayaran','SKU Induk','Nama Produk','Nomor Referensi SKU','Nama Variasi',
       'Harga Awal','Harga Setelah Diskon','Jumlah','Returned quantity','Total Harga Produk','Total Diskon','Diskon Dari Penjual',
       'Diskon Dari Shopee','Berat Produk','Jumlah Produk di Pesan','Total Berat','Voucher Ditanggung Penjual','ditanggung penjual no dup',
       'Cashback Koin','Voucher Ditanggung Shopee','Paket Diskon','Paket Diskon (Diskon dari Shopee)','Paket Diskon (Diskon dari Penjual)',
       'Potongan Koin Shopee','Diskon Kartu Kredit','Ongkos Kirim Dibayar oleh Pembeli','Estimasi Potongan Biaya Pengiriman','Ongkos Kirim Pengembalian Barang',
       'Total Pembayaran','total no dup','Perkiraan Ongkos Kirim','Catatan dari Pembeli','Catatan','Username (Pembeli)','Nama Penerima',
       'No. Telepon','Alamat Pengiriman','Kota/Kabupaten','Provinsi','Waktu Pesanan Selesai','source',
 ]
groupdate = revenue.with_columns([pl.col('Waktu Pembayaran Dilakukan').dt.date().alias('Group_date').cast(pl.Utf8),
                                  pl.col('Waktu Pembayaran Dilakukan').dt.strftime(format='%m-%d').alias('month_days').cast(pl.Utf8),
                                  pl.col('Waktu Pembayaran Dilakukan').dt.week().alias('week_of_q4')]).filter(pl.col('Group_date').str.contains(pattern='2024-10'))

In [269]:
groupdate.write_parquet('for_dashboard.parquet',compression='zstd')

# executive

In [243]:
groupdate.group_by('Group_date').agg(pl.col('total no dup').sum()).sort('Group_date').plot.line(x='Group_date',y='total no dup')

alt.Chart(...)

### insight 

if we penalize x factor like event 10. 10 shopee the first 2 week i goes rally start from 4th day of month before they falling off

In [244]:
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [245]:
corr = groupdate.group_by('Group_date').agg([pl.col('total no dup').sum().alias('total rev'),pl.col('ditanggung penjual no dup').sum().alias('total discount')]).sort(by='Group_date')

corr.plot.scatter(y='total rev',x='total discount')

alt.Chart(...)

In [246]:
var1 = groupdate['total no dup'].to_numpy()
var2 = groupdate['ditanggung penjual no dup'].to_numpy()

corr , pval = stats.spearmanr(var1,var2)

print(f'correlation value"s are {corr} and pvalue is {pval:.5f}')

correlation value"s are 0.6117563435825989 and pvalue is 0.00000


In [247]:
corr*corr*100

np.float64(37.424582391355074)

### insight

whenever our store gave discount on the product the revenue tends to follow that eg. when our revenue is 1000000 around 37% (its squared of correlation value) of it can be achive because of discount strategies move by our company

In [248]:
groupdate.with_columns([(pl.col('Harga Awal')*pl.col('Jumlah')).alias('total_harga_asli')]).group_by('week_of_q4').agg([pl.col('total no dup').sum(),
                                                                                                                      pl.col('Harga Awal').sum(),
                                                                                                                      pl.col('total_harga_asli').sum()]).sort(by='week_of_q4').with_columns([
                                                                                                                          (pl.col('total no dup')-pl.col('total_harga_asli')).alias('profit')
                                                                                                                          ])

week_of_q4,total no dup,Harga Awal,total_harga_asli,profit
i8,i64,i64,i64,i64
40,1154423661,1310837000,1391409000,-236985339
41,1895433076,2241727000,2347688500,-452255424
42,1401842080,1620867500,1769321000,-367478920
43,1190948628,1481680500,1558440500,-367491872
44,784957002,996388000,1046343500,-261386498


In [249]:
groupdate

No. Pesanan,Status Pesanan,Alasan Pembatalan,Status Pembatalan/ Pengembalian,No. Resi,Opsi Pengiriman,Antar ke counter/ pick-up,Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan),Waktu Pengiriman Diatur,Waktu Pesanan Dibuat,Waktu Pembayaran Dilakukan,Metode Pembayaran,SKU Induk,Nama Produk,Nomor Referensi SKU,Nama Variasi,Harga Awal,Harga Setelah Diskon,Jumlah,Returned quantity,Total Harga Produk,Total Diskon,Diskon Dari Penjual,Diskon Dari Shopee,Berat Produk,Jumlah Produk di Pesan,Total Berat,Voucher Ditanggung Penjual,ditanggung penjual no dup,Cashback Koin,Voucher Ditanggung Shopee,Paket Diskon,Paket Diskon (Diskon dari Shopee),Paket Diskon (Diskon dari Penjual),Potongan Koin Shopee,Diskon Kartu Kredit,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Ongkos Kirim Pengembalian Barang,Total Pembayaran,total no dup,Perkiraan Ongkos Kirim,Catatan dari Pembeli,Catatan,Username (Pembeli),Nama Penerima,No. Telepon,Alamat Pengiriman,Kota/Kabupaten,Provinsi,Waktu Pesanan Selesai,source,Group_date,month_days,week_of_q4
str,str,str,null,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],str,null,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,null,null,str,str,str,str,str,str,datetime[ms],str,str,str,i8
"""241001CUKMTPHP""","""Selesai""",null,null,"""GK-11-784428416""","""Instant-GoSend Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 10:58:00,2024-10-01 07:32:00,2024-10-01 07:32:00,"""ShopeePay""",null,"""UGREEN Adaptor Kepala Charger …","""10220""","""ADP 20W MINI WHITE""",119000,119000,1,0,119000,0,0,0,"""250 gr""",1,"""250 gr""",0,0,0,2380,"""N""",0,0,0,0,0,18000,0,118620,118620,18000,null,null,"""nfiata""","""N***a""","""******53""","""Jl. Batang Hari No.6, RT.5/RW.…","""KOTA JAKARTA PUSAT""","""DKI JAKARTA""",2024-10-09 04:29:00,"""10_2024-oct (1).xlsx""","""2024-10-01""","""10-01""",40
"""241001CTY0TPCF""","""Selesai""",null,null,"""2251004833009152369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:28:00,2024-10-01 07:21:00,2024-10-01 07:21:00,"""ShopeePay""",null,"""UGREEN Charger iPhone MFi USB …","""50460-60161""","""1SET 10W WHITE NYLON""",212500,212500,1,0,212500,0,0,0,"""200 gr""",1,"""200 gr""",0,0,0,21250,"""N""",0,0,0,0,0,20000,0,211250,211250,20000,null,null,"""zahrah.mutiara""","""Z******a""","""******33""","""Jalan Kp Rawa Sawah No.7, RT.2…","""KOTA JAKARTA PUSAT""","""DKI JAKARTA""",2024-10-01 17:14:00,"""10_2024-oct (1).xlsx""","""2024-10-01""","""10-01""",40
"""241001D0D47A0N""","""Selesai""",null,null,"""2251004910424064369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:39:00,2024-10-01 08:58:00,2024-10-01 08:59:00,"""Online Payment""",null,"""UGREEN USB Hub 10 in 1 Type C …","""15601""","""15601 10 IN 1""",490000,490000,3,0,1470000,0,0,0,"""900 gr""",3,"""900 gr""",0,0,0,0,"""N""",0,0,0,0,0,28000,0,1486400,1486400,28000,null,null,"""hestisusanti20""","""H******i""","""******14""","""Ruko Bona Gabe, Jalan Jatinega…","""KOTA JAKARTA TIMUR""","""DKI JAKARTA""",2024-10-07 15:07:00,"""10_2024-oct (1).xlsx""","""2024-10-01""","""10-01""",40
"""241001D04YUDH1""","""Selesai""",null,null,"""2251005068414464369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:40:00,2024-10-01 08:54:00,2024-10-01 08:54:00,"""ShopeePay""",null,"""UGREEN Kabel Data MFI iPhone 5…","""60158""","""60158 2M""",175000,175000,1,0,175000,0,0,0,"""200 gr""",1,"""200 gr""",2500,2500,0,13800,"""N""",0,0,0,0,0,18000,0,162400,162400,18000,null,null,"""indahhhs.61""","""w******l""","""******63""","""Warung Pak Haji Alek/ Bang Zul…","""KOTA JAKARTA BARAT""","""DKI JAKARTA""",2024-10-01 23:49:00,"""10_2024-oct (1).xlsx""","""2024-10-01""","""10-01""",40
"""241001CUY3WU7E""","""Selesai""",null,null,"""2251005018542080369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:46:00,2024-10-01 07:39:00,2024-10-01 07:39:00,"""ShopeePay""",

### want to dig about what make after hitting mid months slowsdown

In [ ]:
pl.Config.set_tbl_rows(50)
ralies = groupdate.filter(pl.col('month_days')<= '10-15')
slowsdown = groupdate.filter(pl.col('month_days') > '10-15')
investigate1 = (ralies
 .group_by(['week_of_q4', 'Nomor Referensi SKU'])
 .agg([
     pl.len(),
     pl.col('total no dup').sum().alias('revenue')
 ])
 .sort(by=['week_of_q4', 'revenue'], descending=[False, True])
 .group_by('week_of_q4')
 .head(5)  # This will get top 5 SKUs by revenue for each week
)

pl.Config.set_tbl_rows(50)
investigate2 = (slowsdown
 .group_by(['week_of_q4', 'Nomor Referensi SKU'])
 .agg([
     pl.len(),
     pl.col('total no dup').sum().alias('revenue')
 ])
 .sort(by=['week_of_q4', 'revenue'], descending=[False, True])
 .group_by('week_of_q4')
 .head(5)  # This will get top 5 SKUs by revenue for each week
)

fig1 = px.bar(investigate1, x='week_of_q4', y='revenue', 
              color='Nomor Referensi SKU', title='Rallies Top 5 SKUs')
fig2 = px.bar(investigate2, x='week_of_q4', y='revenue', 
              color='Nomor Referensi SKU', title='Slowdown Top 5 SKUs')

# Combined figure can be created with:
combined_fig = go.Figure(data=fig1.data + fig2.data)
combined_fig.show()

### insight investigate

#### context
- i do split data for investigate what causing after hit 15th oct date sales goes slows down.
- i do grouping sales into several weeks during oct sales run, then seeing its what are the top 5 sku each week by its sales generated sorted from big to small.

#### the insight:
- SKU line 50799 losing its grip when hit the last 2 week of oct 2024 causing after mid period of oct 2024 sales slows down
- SKU 60450 remain stable among the oct sales run also for SKU 15324 helping sales performance isn't too drop
- SKU 90664-60759 helping sales growth before 15th oct
- somehow shopee event on 10.10 sales doesn't affect that significant compare to the first week while the growth are shown at some SKU include the SKU 50799 and 60450
- other SKU [ 15324 , 25208 , 50581 , 60450-10493 , 15324 , 20476 , 25742 , 15086 , 25683] is kinda come and go product, shows it only appear 1 times during those several week group , why the more staple product are atleast 2 times shown in top 5 in every several weekly grouping ( kinda thank to that because it give support to the overall sales)

#### suggestion based on that
- keep product SKU 50799 still available on-live sales to minimize loose on that SKU sales-run

In [267]:
disc = groupdate.with_columns(
    ((1 - (pl.col('Harga Setelah Diskon') / pl.col('Harga Awal'))) * 100).alias('pct_discount')
)

'''
discount UNDER 10% 5278469198
discount UNDER 9% 5276966586
discount UNDER 8% 5270197515
discount UNDER 7% 5267292686
discount UNDER 6% 5264721901
discount UNDER 5% 4727537903
'''
print(f'discount UNDER 6% {disc.filter(pl.col('pct_discount')<=6)['total no dup'].sum():,.2f}')
print(f'discount ABOVE 6% {disc.filter(pl.col('pct_discount')>6)['total no dup'].sum():,.2f}')
print(f'total {disc['total no dup'].sum():,.2f}')

less6 = disc.filter(pl.col('pct_discount')<=6)
qqq = less6.group_by('Nomor Referensi SKU').agg(pl.col('total no dup').sum()).sort('total no dup',descending=True).with_columns(pl.col('total no dup').cum_sum().alias('cumul'))
fig = px.bar(qqq.head(10), 
             x='Nomor Referensi SKU', 
             y='total no dup',
             category_orders={"Nomor Referensi SKU": qqq['Nomor Referensi SKU'].to_list()},
             color='total no dup',
             )
fig.update_traces(
    marker_color=px.colors.qualitative.T10,
    hovertemplate='Total: %{y}<br>SKU: %{x}<extra></extra>'
)
fig.show()
# fig.update_traces(marker_color = px.colors.qualitative.T10)
# base on situation above 

discount UNDER 6% 5,264,721,901.00
discount ABOVE 6% 1,162,882,546.00
total 6,427,604,447.00


#### insight 

here are the sku list that should be shown in top performance in each week:

10133-60161 // 10191 // '15324' // 15326 // '25683' // '25742' // '50799' // 60126 // '60450' // 90664 <br>

the situation was only 5 + 1 (90664 if consider the same as 90664-60759) shown at number that has single quotes , this indicating some kinda inconsintency on sales side of that sku line 

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#### going to forecast for the top 3 SKU above using WMA in excel combine with MRP stuff
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


## corelation between jumlah sold amount with its price tag during rallies and slowdown phase

In [254]:
awalrally = ralies.group_by('Harga Awal').agg(pl.col('Jumlah').sum()).plot.scatter(x='Jumlah',y='Harga Awal')
modreally = ralies.group_by('Harga Setelah Diskon').agg(pl.col('Jumlah').sum()).plot.scatter(x='Jumlah',y='Harga Setelah Diskon')
import altair as alt

alt.concat(awalrally,modreally, spacing=5)


alt.ConcatChart(...)

In [255]:
awalslow = slowsdown.group_by('Harga Awal').agg(pl.col('Jumlah').sum()).plot.scatter(x='Jumlah',y='Harga Awal')
modslow = slowsdown.group_by('Harga Setelah Diskon').agg(pl.col('Jumlah').sum()).plot.scatter(x='Jumlah',y='Harga Setelah Diskon')

alt.concat(awalslow,modslow)

alt.ConcatChart(...)

### insight investigate

both period either rallies and slowingdown period harga awal doesnt the cause that 2 period happen

### deep dive on how big the discount is

In [256]:
# .plot.scatter(y='Harga Awal',x='Jumlah',color='pct_discount',size='total no dup')
# .plot.scatter(y='Harga Awal',x='Jumlah',color='pct_discount',size='total no dup')

In [257]:
groupdate.filter(pl.col('No. Pesanan')=='241005R9V7WERX')

No. Pesanan,Status Pesanan,Alasan Pembatalan,Status Pembatalan/ Pengembalian,No. Resi,Opsi Pengiriman,Antar ke counter/ pick-up,Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan),Waktu Pengiriman Diatur,Waktu Pesanan Dibuat,Waktu Pembayaran Dilakukan,Metode Pembayaran,SKU Induk,Nama Produk,Nomor Referensi SKU,Nama Variasi,Harga Awal,Harga Setelah Diskon,Jumlah,Returned quantity,Total Harga Produk,Total Diskon,Diskon Dari Penjual,Diskon Dari Shopee,Berat Produk,Jumlah Produk di Pesan,Total Berat,Voucher Ditanggung Penjual,ditanggung penjual no dup,Cashback Koin,Voucher Ditanggung Shopee,Paket Diskon,Paket Diskon (Diskon dari Shopee),Paket Diskon (Diskon dari Penjual),Potongan Koin Shopee,Diskon Kartu Kredit,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Ongkos Kirim Pengembalian Barang,Total Pembayaran,total no dup,Perkiraan Ongkos Kirim,Catatan dari Pembeli,Catatan,Username (Pembeli),Nama Penerima,No. Telepon,Alamat Pengiriman,Kota/Kabupaten,Provinsi,Waktu Pesanan Selesai,source,Group_date,month_days,week_of_q4
str,str,str,null,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],str,null,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,null,null,str,str,str,str,str,str,datetime[ms],str,str,str,i8
"""241005R9V7WERX""","""Selesai""",null,null,"""SPXID04868352221A""","""Reguler (Cashless)-SPX Standar…","""Antar ke Counter""",2024-10-07 23:59:00,2024-10-07 18:46:00,2024-10-05 20:47:00,2024-10-05 20:47:00,"""COD (Bayar di Tempat)""",null,"""UGREEN Mouse pad Gaming PU Lea…","""90563-90855""","""90563+Mouse BT""",197500,197500,1,0,197500,0,0,0,"""200 gr""",3,"""500 gr""",0,0,0,40000,"""N""",0,0,0,0,0,61999,0,257568,257568,61999,null,null,"""m.yussril""","""M******l""","""******16""","""Toko NN tex, Jalan Jalan Medan…","""KAB. BIREUEN""","""NANGGROE ACEH DARUSSALAM (NAD)""",2024-10-12 00:35:00,"""10_2024-oct (2).xlsx""","""2024-10-05""","""10-05""",40
"""241005R9V7WERX""","""Selesai""",null,null,"""SPXID04868352221A""","""Reguler (Cashless)-SPX Standar…","""Antar ke Counter""",2024-10-07 23:59:00,2024-10-07 18:46:00,2024-10-05 20:47:00,2024-10-05 20:47:00,"""COD (Bayar di Tempat)""",null,"""UGREEN OTG Type C To USB 3.0 C…","""50283""","""UGREEN 50283""",40000,40000,1,0,40000,0,0,0,"""200 gr""",3,"""500 gr""",0,0,0,40000,"""N""",0,0,0,0,0,61999,0,257568,0,61999,null,null,"""m.yussril""","""M******l""","""******16""","""Toko NN tex, Jalan Jalan Medan…","""KAB. BIREUEN""","""NANGGROE ACEH DARUSSALAM (NAD)""",2024-10-12 00:35:00,"""10_2024-oct (2).xlsx""","""2024-10-05""","""10-05""",40
"""241005R9V7WERX""","""Selesai""",null,null,"""SPXID04868352221A""","""Reguler (Cashless)-SPX Standar…","""Antar ke Counter""",2024-10-07 23:59:00,2024-10-07 18:46:00,2024-10-05 20:47:00,2024-10-05 20:47:00,"""COD (Bayar di Tempat)""",null,"""UGREEN Converter Adapter Mini …","""20137""","""UGREEN 20137""",175000,47500,1,0,47500,127500,127500,0,"""100 gr""",3,"""500 gr""",0,0,0,40000,"""N""",0,0,0,0,0,61999,0,257568,0,61999,null,null,"""m.yussril""","""M******l""","""******16""","""Toko NN tex, Jalan Jalan Medan…","""KAB. BIREUEN""","""NANGGROE ACEH DARUSSALAM (NAD)""",2024-10-12 00:35:00,"""10_2024-oct (2).xlsx""","""2024-10-05""","""10-05""",40
